# Datacamp: Michelin restaurant classification
Authors : *Quentin Garsault, Charles Cuvillier, Lucas Selini, Eric Patarin*

## Table of Contents
* [Introduction](#introduction)
* [The dataset](#dataset)
* [Requirements](#requirements)
* [Data exploration](#exploration)
* [Base model](#base_model)

## Introduction <a class="anchor" id="introduction"></a>
Write an introduction

## The dataset <a class="anchor" id="dataset"></a>

For this challenge, the data were first preprocessed and then split in order to preserve a private test set on which to evaluate the models on the servers. This leaves XXXX observations in the public train set and XXXX observations in the public test set. 

Preprocessing steps before splitting the data:


Glossary of the terms used in the dataset:

## Requirements <a class="anchor" id="requirements"></a>

In [ ]:
import pandas as pd
import numpy as np
import missingno as msno
import matplotlib.pyplot as plt
import seaborn as sns
import os

from sklearn.model_selection import StratifiedGroupKFold
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.base import BaseEstimator
from sklearn.pipeline import Pipeline,make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB

# workflow libraires
from sklearn.base import is_classifier
from sklearn.utils import _safe_indexing
# from ..utils.importing import import_module_from_source

import rampwf as rw

## Download Data

## Data Exploration <a class="anchor" id="exploration"></a>

In [ ]:
problem_title = "Classification de restaurant au guide Michelin"

# Correspondence between categories and int8 categories
# Mapping int to categories
int_to_cat = {
   1 : 'ONE STAR',
   2 : 'TWO STARS',
   3 : 'THREE STARS',
}

# Mapping categories to int
cat_to_int = {v: k for k, v in int_to_cat.items()}


def _get_data(path=".", split="train", cat_to_int = cat_to_int):
    # Load data from csv files into pd.DataFrame

    data_df = pd.read_csv(os.path.join(path, "data", split + ".csv"))
    data_df["SampleID"] = data_df["SampleID"].astype("category")
    SampleID = np.array(data_df["SampleID"].cat.codes)

    majors = [

    ]

    traces = [

    ]

    X_majors = data_df.loc[:, majors]
    X_traces = data_df.loc[:, traces]
    X_df = pd.concat([X_majors, X_traces], axis=1)

    # X_df["groups"] = SampleID.tolist() # présent dans la version de base (baisse la précision)
    X = X_df

    # labels
    y = np.array(data_df["Event"].map(cat_to_int).fillna(-1).astype("int8"))

    return X, y

## Base model <a class="anchor" id="base_model"></a>